In [11]:
import pandas as pd
import re
from datetime import datetime
import yfinance as yf
import os 

DATA_DIR = 'dsa_airflow/data'
stock_file = 'raw_stock.csv'

In [35]:
# get historical market data
# https://stackoverflow.com/questions/44225771/scraping-historical-data-from-yahoo-finance-with-python

# create yf obj
tsla = yf.Ticker("TSLA")

# get 1mo history
hist = tsla.history(period="1mo")

# create dataframe
df = pd.DataFrame(hist)

# reset index for date column
df = df.reset_index()

# # write to file
# df.to_csv(os.path.join(DATA_DIR, 'raw_stock.csv'), header=True, index=False)
# df = pd.read_csv(os.path.join(DATA_DIR, stock_file), header=0)
# print("saved to file")

In [36]:
df.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-02-08 00:00:00-05:00,196.100006,203.000000,194.309998,201.289993,180673600,0.0,0.0
1,2023-02-09 00:00:00-05:00,207.779999,214.000000,204.770004,207.320007,215431400,0.0,0.0
2,2023-02-10 00:00:00-05:00,202.229996,206.199997,192.889999,196.889999,204193800,0.0,0.0
3,2023-02-13 00:00:00-05:00,194.419998,196.300003,187.610001,194.639999,172475500,0.0,0.0
4,2023-02-14 00:00:00-05:00,191.940002,209.820007,189.440002,209.250000,216455700,0.0,0.0


In [37]:
# lower cols name
df.columns = df.columns.str.lower()

# drop columns
df = df.drop(columns=["dividends", "stock splits"]) 

df.head(5)

,date,open,high,low,close,volume
0,2023-02-08 00:00:00-05:00,196.100006,203.000000,194.309998,201.289993,180673600
1,2023-02-09 00:00:00-05:00,207.779999,214.000000,204.770004,207.320007,215431400
2,2023-02-10 00:00:00-05:00,202.229996,206.199997,192.889999,196.889999,204193800
3,2023-02-13 00:00:00-05:00,194.419998,196.300003,187.610001,194.639999,172475500
4,2023-02-14 00:00:00-05:00,191.940002,209.820007,189.440002,209.250000,216455700


In [38]:
# only save the date 
df['date'] = df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

df.head(5)

,date,open,high,low,close,volume
0,2023-02-08,196.100006,203.000000,194.309998,201.289993,180673600
1,2023-02-09,207.779999,214.000000,204.770004,207.320007,215431400
2,2023-02-10,202.229996,206.199997,192.889999,196.889999,204193800
3,2023-02-13,194.419998,196.300003,187.610001,194.639999,172475500
4,2023-02-14,191.940002,209.820007,189.440002,209.250000,216455700


In [39]:
df.dtypes

date       object
open      float64
high      float64
low       float64
close     float64
volume      int64
dtype: object

In [40]:
df[['year', 'month', 'day']] = df.date.str.split("-", expand=True)

df = df.astype({'date': 'datetime64[ns]', 'day': 'int64', 'month': 'int64', 'year':'int64'}) 

df.dtypes

date      datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume             int64
year               int64
month              int64
day                int64
dtype: object

In [43]:
df.head(20)

,date,open,high,low,close,volume,year,month,day
0,2023-02-08,196.100006,203.000000,194.309998,201.289993,180673600,2023,2,8
1,2023-02-09,207.779999,214.000000,204.770004,207.320007,215431400,2023,2,9
2,2023-02-10,202.229996,206.199997,192.889999,196.889999,204193800,2023,2,10
3,2023-02-13,194.419998,196.300003,187.610001,194.639999,172475500,2023,2,13
4,2023-02-14,191.940002,209.820007,189.440002,209.250000,216455700,2023,2,14
5,2023-02-15,211.759995,214.660004,206.110001,214.240005,182108600,2023,2,15
6,2023-02-16,210.779999,217.649994,201.839996,202.039993,229586500,2023,2,16
7,2023-02-17,199.990005,208.440002,197.500000,208.309998,213080200,2023,2,17
8,2023-02-21,204.990005,209.710007,197.220001,197.369995,180018600,2023,2,21
9,2023-02-22,197.929993,201.990005,191.779999,200.860001,191828500,2023,2,22


In [29]:
from google.cloud import bigquery

In [30]:
PROJECT_ID = 'deb-01-372112'
DATASET_ID = 'SIA_Stocks'

# create bigquery client
client = bigquery.Client()

# set dataset_id
dataset_id = "{}.SIA_Stocks".format(client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)

print("Created dataset {}.{}".format(client.project, dataset.dataset_id))


Created dataset deb-01-372112.SIA_Stocks


In [31]:
def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: bigquery.SchemaField,
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish


In [41]:
# create table
table_id = "{}.SIA_Stocks.historial_stock_prices".format(client.project)

schema = [
    bigquery.SchemaField("date", "date", mode="REQUIRED"),
    bigquery.SchemaField("open", "float", mode="REQUIRED"),
    bigquery.SchemaField("high", "float", mode="REQUIRED"),
    bigquery.SchemaField("low", "float", mode="REQUIRED"),
    bigquery.SchemaField("close", "float", mode="REQUIRED"),
    bigquery.SchemaField("volume", "integer", mode="REQUIRED"),
    bigquery.SchemaField("year", "integer", mode="REQUIRED"),
    bigquery.SchemaField("month", "integer", mode="REQUIRED"),
    bigquery.SchemaField("day", "integer", mode="REQUIRED"),
]

table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table, exists_ok=True)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table deb-01-372112.SIA_Stocks.historial_stock_prices


In [42]:
TABLE_ID = 'historial_stock_prices'

table_name = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"
schema = schema
load_table(df, client, table_name, schema)

In [44]:
df.dtypes

date      datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume             int64
year               int64
month              int64
day                int64
dtype: object

In [ ]:
df.to_csv(os.path.join(DATA_DIR, 'stock.csv'), header=True, index=False)